## Importar datos

In [16]:
import pandas as pd
from scipy.stats import f, norm, t
import numpy as np

In [17]:
datos = pd.read_csv("DatosTrabajo2EAE20242.txt")

#### Punto 3- IC del 93% confianza: 
Construir en Python un IC para al 93% de confianza necesario para responder:
¿Puede afirmarse que la proporci´on de colegios privados es m´ınimo del 50%? No
asuma normalidad, justifique su respuesta.

#### Para responder esta pregunta, es necesario obtener al nivel de confianza del 93% los intervalos de confianza para la proporcion poblacional, a partir de los estadisticos muestrales. Para ello se obtiene la proporcion muestral de colegios publicos y privados

In [18]:
### tamaño de la muestra y proporciones
n = datos.shape[0]
publico, privado =(datos["NATURALEZA"].value_counts().values / n)
print(f"instituciones públicas: {publico}\n"
      f"instituciones privadas: {privado}\n"
      f"Tamaño de la muestra: {n}")

instituciones públicas: 0.811289429994466
instituciones privadas: 0.18871057000553404
Tamaño de la muestra: 1807


#### Con los estadisticos muestrales se calculan los valores de la distribucion teorica para construir los intervalos de confianza

In [19]:
### Distribucion z-estandar teorica
tolerancia = 0.07
confianza = 1- (tolerancia/2)
valor_dist_teorica = norm.ppf(confianza)

## Hipotesis: Porporcion de colegios privados es minimo 50%
Se plantea la hipotesis de que = Publicos >= 50 %
para evaluar esto, se puede hallar los intervalos de confianza para la proporcion poblacional y evaluar si el valor minimo del intervalo contiene 0.5

In [20]:
print(f"IC inferior : {privado - valor_dist_teorica* np.sqrt((privado * publico / n))}, IC Superior : {privado + valor_dist_teorica* np.sqrt((privado * publico / n))}")

IC inferior : 0.17203257924160517, IC Superior : 0.2053885607694629


#### Respuesta punto 3: Se concluye la proporcion de colegios privados no es minimo 50%, ya que, se estima que la proporcion de colegios privados en la poblacion es de 17.2% a 20.53%

#### Punto 4 - IC del 90% confianza.
¿Se puede afirmar que el resultado medio obtenido por los colegios en Lectura Cr´ıtica
es mayor en los colegios de jornada completa que los que prestan servicio en
jornadas diferentes? Construir en Python un IC para al 90% de confianza asumiendo
normalidad de las variables involucradas, justifique su respuesta. 

In [ ]:
## Renombrar jornadas por completa y extaraordinarias
# Crear nueva columna con jornadas agrupadas
datos["jornada_p4"] = datos["JORNADA"].replace({
    "COMPLETA": "COMPLETA",
    "MANANA": "EXTRAORDINARIA",
    "UNICA": "EXTRAORDINARIA",
    "FIN DE SEMANA": "EXTRAORDINARIA",
    "NOCTURNA": "EXTRAORDINARIA",
    "TARDE": "EXTRAORDINARIA"
})
datos["jornada_p4"].value_counts()

jornada_p4
EXTRAORDINARIA    1112
COMPLETA           695
Name: count, dtype: int64

In [ ]:
### Se extraen los valores muestrales: media, varainza y tamaños de las muestras.
media_Completa = datos.query("jornada_p4 == 'COMPLETA'")["PROMLECT"].mean()
media_Extra = datos.query("jornada_p4 == 'EXTRAORDINARIA'")["PROMLECT"].mean()
var_Completa = datos.query("jornada_p4 == 'COMPLETA'")["PROMLECT"].var()
var_Extra = datos.query("jornada_p4 == 'EXTRAORDINARIA'")["PROMLECT"].var()
m, n = datos["jornada_p4"].value_counts().values

##### Para validar si hay diferencias en las medias, asuminedo normalidad de las variables y que en este caso solo se sabe la varinza muestral y no poblacional. se debe chequear si las varianzas de las dos poblaciones son iguales, mediante una prueba. una vez se pruebe que son diferentes o iguales, se usará diferente valor para el calculo del estadistico t y calcular los IC poblacionales

In [41]:
### Revisar si las varianzas poblacionales son iguales o no

f_superior = f.ppf((1-0.025), dfn = n-1 , dfd = m-1)
f_inferior = f.ppf((0.025), dfn = n-1 , dfd = m-1)


(var_Completa**2 / var_Extra**2)*f_superior, (var_Completa**2 / var_Extra**2)*f_inferior 

print(f"IC inferior : {(var_Completa**2 / var_Extra**2)*f_inferior}, IC Superior : {(var_Completa**2 / var_Extra**2)*f_superior }")
# Si contiene al 1 son iguales las varianzas. en este caso contien al uno son iguales

IC inferior : 3.327250308514971, IC Superior : 4.351647667142516


### Como los intervalos de confianza no contienen el 1. entonces, las varianzas son diferentes

Dada las diferencias en cuanto a las varianzas, se el valor de la distribucion teorica t-student con v como parametro

In [37]:
nuemrador = ((var_Completa / n)+(var_Extra/m))**2
denominador_parte_1 = (var_Completa/n)**2
denominador_parte_2 = (var_Extra/m)**2
denominador = (denominador_parte_1 / (n-1)) + (denominador_parte_2/(m-1))
v = round(nuemrador/denominador)

In [38]:
valor_dist_teorica = t.ppf(0.90, v)
valor_dist_teorica

np.float64(1.2822965849550738)

In [44]:
diferencia = media_Completa - media_Extra
print(f"IC inferior : {diferencia - valor_dist_teorica*np.sqrt((var_Completa/n)+(var_Extra/m))}, IC Superior : {diferencia + valor_dist_teorica*np.sqrt((var_Completa/n)+(var_Extra/m))}")

IC inferior : -0.5899527522716241, IC Superior : 0.34930527025722863


#### La idea a evaluar es si los colegios con jornada completa tienen mayores niveles de lectura critica que los colegios de jornada extraordinaria. para evaluar esto es necesario que los intervalos sean estrictamente positivos, dada la hipotesis evaluada: 
#### P_completa - P_extraordinaria = 0
#### no obstante el IC calculado, no evidencia esto, y la verdadera diferencia poblacional podria estar entre -0.5899 , y 0.3493